In [119]:
import pandas as pd
import folium

filename = "행정구역주민등록인구.csv"  

# CSV 파일 읽기
df = pd.read_csv(filename, encoding='ANSI') #이거 안쓰면 한글 안읽힘

df.head()

# 필요한 열 선택 
data = df.iloc[2:20, 0:14]

# 컬럼생성
data.columns = ['행정구역','총인구수','1997_20','1997_25','1997_30',
                '2000_20','2000_25','2000_30',
                '2010_20','2010_25','2010_30',
                '2023_20','2023_25','2023_30',]  #컬럼생성

data=data.drop('총인구수',axis=1)   #총인구는 데이터 작업에 필요치 않으니 제거
data[['1997_20','1997_25','1997_30',
                '2000_20','2000_25','2000_30',
                '2010_20','2010_25','2010_30',
                '2023_20','2023_25','2023_30']]=data[['1997_20','1997_25','1997_30',
                '2000_20','2000_25','2000_30',
                '2010_20','2010_25','2010_30',
                '2023_20','2023_25','2023_30']].apply(pd.to_numeric)
data.loc[:,'1997_총 합']=data.loc[:,'1997_20':'1997_30'].sum(axis=1)
data.loc[:,'2000_총 합']=data.loc[:,'2000_20':'2000_30'].sum(axis=1)
data.loc[:,'2010_총 합']=data.loc[:,'2010_20':'2010_30'].sum(axis=1)
data.loc[:,'2023_총 합']=data.loc[:,'2023_20':'2023_30'].sum(axis=1)

# 위도경도 데이터 입력
city_cords = {
    '서울특별시': [37.5665, 126.9780],
    '부산광역시': [35.1796, 129.0756],
    '경기도': [37.3066, 127.0606],
    '세종특별자치시':[36.5674, 127.2575],
    '대구광역시': [35.8290, 128.5659],
    '인천광역시': [37.4582, 126.7036],
    '광주광역시': [35.1565, 126.8357],
    '대전광역시': [36.3757, 127.3975],
    '울산광역시': [35.5931, 129.2475],
    '경기도': [37.4831, 127.5203],
    '강원특별자치도': [37.8558, 128.6005],
    '충청북도': [37.0547, 127.7058],
    '충청남도': [36.7961, 126.8144],
    '전북특별자치도': [35.7620, 127.1475],
    '전라남도': [34.9565, 126.9852],
    '경상북도': [36.2704, 128.8968],
    '경상남도': [35.5443, 128.2264],
    '제주특별자치도': [33.4439, 126.8030]
}


In [121]:
map_center = [37.5665, 126.9780]    #맵 시작 위치 설정
m = folium.Map(location=map_center, zoom_start=7)   #맵 줌 사이즈 설정

In [122]:
data['latitude'] = data['행정구역'].apply(lambda x: city_cords.get(x, [0, 0])[0])   #위도 데이터    
data['longitude'] = data['행정구역'].apply(lambda x: city_cords.get(x, [0, 0])[1])  #경도 데이터
data['ratio'] = data['2023_총 합']   #반지름 데이터

In [123]:
years=[1997,2000,2010,2023]

In [124]:
# 최대 인구 수 구하기 (상대적인 크기 생성을 위해)
max_population = data[['1997_총 합', '2000_총 합', '2010_총 합','2023_총 합']].max().max().max()

In [125]:
for year in years:
    m = folium.Map(location=[35.7665, 126.9780], zoom_start=7) #기본 위도 경도
    
    for index, row in data.iterrows():
        total_population = float(row[f'{year}_총 합'])
        all_radius = total_population / float(max_population) * 150 # 곱하는 숫자가 작을 수록 원 사이즈 감소
        
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=all_radius,
            popup=f"{total_population}<br>총인구수: {total_population}",
            color='red',
            fill=True,
            fill_color='red'
        ).add_to(m)
    
    # 년도에 맞게 HTML 파일로 저장
    m.save(f"map_{year}.html")